In [1]:
import svgBlockLib

from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False, show_parent=False)

enable_replay(False, False)
show_object = replay

cv = open_viewer("3dViewer", anchor="right", height=1024)

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
blk = svgBlockLib.blkLibrary()

In [ ]:
#Import from file. Could be web or others in the future
#fileName = 'noun-gutenberg-press-86670-cleaned.svg'
fileName = 'noun-houseplants-886781-edited.svg'
#fileName = 'noun-ornaments-714185-cleaned.svg'
blk.readSVGFromFile(fileName)

#Parse using svgpathtools and 'return' paths and attribs.
#Could probably have some checks to make sure paths exist and are closed?
blk.parseSVG()

#See how many paths were parsed
print('Paths Found: %s' % format(blk.pathCount))

#Skip paths. Helpful for testing or omitting paths. 
#Needs to be tested
#blk.skipPathNumber.append([100,200])

#Do the actual translation from 2D to 3D
blk.translate2Dto3D()

In [ ]:
#Add some buffer space around the 2D SVG. Must be >1.0. 
#Must be called before building body
blk.xLenAdj = 1.25
blk.yLenAdj = 1.25

#Draw a box around the 3D SVG to get actual size.
blk.buildBoundingBox()

#With the box built, we can get our height and width
print('Block Width: ', blk.width/blk.scaleBy, "mm")
print('Block Height', blk.height/blk.scaleBy, "mm")

#Fillet outer edges. This value is usually close to the scale set below
# It is updated when setScale is called by multiplying filletAmount by
# the scale set
blk.filletAmount = 0.2

#Before setting scale, update how much to extrude the SVG. This is affected
# when the scale is set below.
blk.neckHeight = 1

#Parsing SVG will provide the Height/Width if listed.
#If the SVG is too large, the 3D block can be scaled. Ideally by factor of 10.
# Must call doMath to generate values
blk.setScale(10)

#With the scale set, start calculating heights and widths
blk.doMath()

#Extrude basic block
blk.buildBody()

In [3]:
'''
This section pertains to the 3D portion of the process

Need to check if hollowBody function needs to create a second 
BoundingBox or if previous box can be used. This could allow for
x/yWallThickness to be defined earlier in the process.

feetCutOut is harder since it uses the X Face to calculate the
different values.
'''

#Adjust the x/y of the hollow box to be cut out. This is a percentage
# of the overal x/y of the block body. Must be < 1. Default: 0.8
# The smaller the number, the larger the hollow area
blk.xhollowPercentage = 0.7
blk.yhollowPercentage = 0.8

#
blk.hollowBody()
blk.createAndCutPyramid()

#Height of the feet cutout can be adjusted. This is in percentage
# of the total block height. This is usually a small number <0.2
blk.feetCutOutPercentage = 0.08
blk.cutFeet()

#Smooth out the block edges a bit
blk.smoothOuterEdges()

#Export to STL file. If scale is anything by 1, it will scale the STL.
# STL only needs to be scaled in the Z direction. By default, it'll
# scale by XYZ.
blk.exportSTL('houseplant')

#https://stackoverflow.com/questions/65241421/scaling-stl-mesh-object-in-python

SVG Width: 700pt
SVG Height: 700pt
Paths Found: 1
Block Width:  436.26875025 mm
Block Height 450.926133875 mm


In [4]:
show_object(blk.base)

Use the multi select box below to select one or more steps you want to examine
